# HISTORY

#### Context
* The main aim of this project is the masking of regions where land is being used in satellite images obtained through Landsat which have a low spatial resolution.

#### Content
* This dataset contains satellite images of 21 classes such as buildings, baseball fields, freeways, etc. The original size of the images is 256x256 pixels. Originally there were 100 images per class. After augmenting each image 4 times the size of each class was brought up to 500 images. This allows for making
* a more robust model.

#### Acknowledgements
* The above dataset was obtained from the UC Merced Dataset
* Dataset: Yi Yang and Shawn Newsam, "Bag-Of-Visual-Words and Spatial Extensions for Land-Use Classification," ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems (ACM GIS), 2010.

# PACKAGES AND LIBRARIES

In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.applications.vgg16 import preprocess_input, decode_predictions
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers
from sklearn.preprocessing import LabelEncoder
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer, SGD
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN, LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)

# PATH & LABEL PROCESS

#### DATA PATH

In [ ]:
Train_Data_Path = Path("../input/landuse-scene-classification/images_train_test_val/train")
Test_Data_Path = Path("../input/landuse-scene-classification/images_train_test_val/test")
Validation_Data_Path = Path("../input/landuse-scene-classification/images_train_test_val/validation")

#### PATH PROCESS

In [ ]:
Train_PNG_Path = list(Train_Data_Path.glob(r"*/*.png"))
Test_PNG_Path = list(Test_Data_Path.glob(r"*/*.png"))
Validation_PNG_Path = list(Validation_Data_Path.glob(r"*/*.png"))

#### LABEL PROCESS

In [ ]:
Train_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Train_PNG_Path))
Test_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Test_PNG_Path))
Validation_PNG_Labels = list(map(lambda x : os.path.split(os.path.split(x)[0])[1],Validation_PNG_Path))

In [ ]:
Labels_List = ["agricultural","agricultural","airplane","baseballdiamond","beach",
 "buildings","chaparral","denseresidential","forest","freeway","golfcourse","harbor",
 "intersection","mediumresidential","mobilehomepark","overpass",
 "parkinglot","river","runway","sparseresidential","storagetanks","tenniscourt"]

In [ ]:
for labels in Labels_List:
    print(f"{labels}: ", Train_PNG_Labels.count(labels))

In [ ]:
for labels in Labels_List:
    print(f"{labels}: ", Test_PNG_Labels.count(labels))

In [ ]:
for labels in Labels_List:
    print(f"{labels}: ", Validation_PNG_Labels.count(labels))

# SERIES STRUCTURE

In [ ]:
Train_PNG_Path_Series = pd.Series(Train_PNG_Path,name="PNG").astype(str)
Test_PNG_Path_Series = pd.Series(Test_PNG_Path,name="PNG").astype(str)
Validation_PNG_Path_Series = pd.Series(Validation_PNG_Path,name="PNG").astype(str)

In [ ]:
print(Train_PNG_Path_Series)
print("---"*20)
print(Test_PNG_Path_Series)
print("---"*20)
print(Validation_PNG_Path_Series)

In [ ]:
Train_PNG_Labels_Series = pd.Series(Train_PNG_Labels,name="CATEGORY")
Test_PNG_Labels_Series = pd.Series(Test_PNG_Labels,name="CATEGORY")
Validation_PNG_Labels_Series = pd.Series(Validation_PNG_Labels,name="CATEGORY")

In [ ]:
print(Train_PNG_Labels_Series.value_counts())
print("---"*20)
print(Test_PNG_Labels_Series.value_counts())
print("---"*20)
print(Validation_PNG_Labels_Series.value_counts())

# DATAFRAME STRUCTURE

In [ ]:
Main_Train_Data = pd.concat([Train_PNG_Path_Series,Train_PNG_Labels_Series],axis=1)
Main_Test_Data = pd.concat([Test_PNG_Path_Series,Test_PNG_Labels_Series],axis=1)
Main_Validation_Data = pd.concat([Validation_PNG_Path_Series,Validation_PNG_Labels_Series],axis=1)

In [ ]:
print(Main_Train_Data.head(-1))
print("---"*20)
print(Main_Test_Data.head(-1))
print("---"*20)
print(Main_Validation_Data.head(-1))

In [ ]:
print(Main_Train_Data["CATEGORY"].value_counts())

#### SHUFFLING

In [ ]:
Main_Train_Data = Main_Train_Data.sample(frac=1).reset_index(drop=True)
Main_Test_Data = Main_Test_Data.sample(frac=1).reset_index(drop=True)
Main_Validation_Data = Main_Validation_Data.sample(frac=1).reset_index(drop=True)

In [ ]:
print(Main_Train_Data.head(-1))
print("---"*20)
print(Main_Test_Data.head(-1))
print("---"*20)
print(Main_Validation_Data.head(-1))

# VISUALIZATION

In [ ]:
plt.style.use("dark_background")

In [ ]:
figure = plt.figure(figsize=(10,10))
x = plt.imread(Main_Train_Data["PNG"][20])
plt.imshow(x)
plt.xlabel(x.shape)
plt.title(Main_Train_Data["CATEGORY"][20])

In [ ]:
figure = plt.figure(figsize=(10,10))
x = plt.imread(Main_Train_Data["PNG"][55])
plt.imshow(x)
plt.xlabel(x.shape)
plt.title(Main_Train_Data["CATEGORY"][55])

In [ ]:
figure = plt.figure(figsize=(10,10))
x = plt.imread(Main_Train_Data["PNG"][1405])
plt.imshow(x)
plt.xlabel(x.shape)
plt.title(Main_Train_Data["CATEGORY"][1405])

In [ ]:
rows = 3
cols = 5

figure,axis = plt.subplots(rows,cols,figsize=(12,12))

for i in range(0,rows):
    for j in range(0,cols):
        axis[i,j].xaxis.set_ticklabels([])
        axis[i,j].yaxis.set_ticklabels([])
        axis[i,j].imshow(plt.imread(Main_Train_Data["PNG"][np.random.randint(1000)]))
plt.show()

In [ ]:
rows = 3
cols = 5

figure,axis = plt.subplots(rows,cols,figsize=(12,12))

for i,ax in enumerate(axis.flat):
    ax.imshow(plt.imread(Main_Train_Data["PNG"][i]))
    ax.set_title(Main_Train_Data["CATEGORY"][i])
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(1,4,figsize=(12,12))

image = Main_Train_Data["PNG"][30]
image = cv2.imread(image)
image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
image_color = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
image_edges = cv2.Canny(image_color,30,200)
image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
axis[0].imshow(image_edges)
axis[1].imshow(image_color)
axis[2].imshow(image_blurred)
axis[3].imshow(image_sharp)


In [ ]:
figure,axis = plt.subplots(1,4,figsize=(12,12))

image_Two = Main_Train_Data["PNG"][2005]
image_Two = cv2.imread(image_Two)
image_blurred_Two = cv2.GaussianBlur(image_Two, (0, 0), 3)
image_color_Two = cv2.cvtColor(image_Two,cv2.COLOR_RGB2GRAY)
image_edges_Two = cv2.Canny(image_color_Two,30,200)
image_sharp_Two = cv2.addWeighted(image_Two, 1.5, image_blurred_Two, -0.5, 0)
axis[0].imshow(image_edges_Two)
axis[1].imshow(image_color_Two)
axis[2].imshow(image_blurred_Two)
axis[3].imshow(image_sharp_Two)

In [ ]:
figure,axis = plt.subplots(1,4,figsize=(12,12))

image_Three = Main_Train_Data["PNG"][300]
image_Four = Main_Train_Data["PNG"][1255]

image_Three = cv2.imread(image_Three)
image_Four = cv2.imread(image_Four)

image_edges_Three = cv2.Canny(image_Three,30,200)
image_edges_Four = cv2.Canny(image_Four,30,200)

contours, _ = cv2.findContours(image_edges_Three,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

contours_Two, _ = cv2.findContours(image_edges_Four,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

cv2.drawContours(image_Three, contours, -1, (0, 255, 0), 1)

cv2.drawContours(image_Four, contours_Two, -1, (0, 0, 255), 1)

axis[0].imshow(image_edges_Three)
axis[1].imshow(image_Three)
axis[2].imshow(image_edges_Four)
axis[3].imshow(image_Four)

In [ ]:
image_Three = Main_Train_Data["PNG"][300]
image_Four = Main_Train_Data["PNG"][1255]
image_Two = Main_Train_Data["PNG"][2005]
image = Main_Train_Data["PNG"][30]

image = cv2.imread(image)
image_Two = cv2.imread(image_Two)
image_Three = cv2.imread(image_Three)
image_Four = cv2.imread(image_Four)

list_image = [image,image_Two,image_Three,image_Four]

ret,thresh = cv2.threshold(image,127,255,cv2.THRESH_BINARY_INV)
ret,thresh_Two = cv2.threshold(image_Two,127,255,cv2.THRESH_BINARY_INV)
ret,thresh_Three = cv2.threshold(image_Three,127,255,cv2.THRESH_BINARY_INV)
ret,thresh_Four = cv2.threshold(image_Four,127,255,cv2.THRESH_BINARY_INV)

thresh_list = [thresh,thresh_Two,thresh_Three,thresh_Four]

fig, axes = plt.subplots(nrows=2,
                         ncols=2,
                         figsize=(20, 20),
                        subplot_kw={'xticks': [], 'yticks': []})


axes[0,0].imshow(thresh_list[0])
axes[0,1].imshow(thresh_list[1])
axes[1,0].imshow(thresh_list[2])
axes[1,1].imshow(thresh_list[3])

In [ ]:
image_Three = Main_Train_Data["PNG"][300]
image_Four = Main_Train_Data["PNG"][1255]
image_Two = Main_Train_Data["PNG"][2005]
image = Main_Train_Data["PNG"][30]

image = cv2.imread(image,0)
image_Two = cv2.imread(image_Two,0)
image_Three = cv2.imread(image_Three,0)
image_Four = cv2.imread(image_Four,0)

list_image = [image,image_Two,image_Three,image_Four]

thresh = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
thresh_Two = cv2.adaptiveThreshold(image_Two,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
thresh_Three = cv2.adaptiveThreshold(image_Three,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
thresh_Four = cv2.adaptiveThreshold(image_Four,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)

thresh_list = [thresh,thresh_Two,thresh_Three,thresh_Four]


fig, axes = plt.subplots(nrows=2,
                         ncols=2,
                         figsize=(20, 20),
                        subplot_kw={'xticks': [], 'yticks': []})

axes[0,0].imshow(thresh_list[0])
axes[0,1].imshow(thresh_list[1])
axes[1,0].imshow(thresh_list[2])
axes[1,1].imshow(thresh_list[3])

In [ ]:
image_Three = Main_Train_Data["PNG"][300]
image_Four = Main_Train_Data["PNG"][1255]
image_Two = Main_Train_Data["PNG"][2005]
image = Main_Train_Data["PNG"][30]

image = cv2.imread(image,0)
image_Two = cv2.imread(image_Two,0)
image_Three = cv2.imread(image_Three,0)
image_Four = cv2.imread(image_Four,0)

list_image = [image,image_Two,image_Three,image_Four]

thresh = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
thresh_Two = cv2.adaptiveThreshold(image_Two,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
thresh_Three = cv2.adaptiveThreshold(image_Three,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
thresh_Four = cv2.adaptiveThreshold(image_Four,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)

thresh_list = [thresh,thresh_Two,thresh_Three,thresh_Four]


fig, axes = plt.subplots(nrows=2,
                         ncols=2,
                         figsize=(20, 20),
                        subplot_kw={'xticks': [], 'yticks': []})

axes[0,0].imshow(thresh_list[0])
axes[0,1].imshow(thresh_list[1])
axes[1,0].imshow(thresh_list[2])
axes[1,1].imshow(thresh_list[3])

In [ ]:
Back_Subt = cv2.createBackgroundSubtractorMOG2()

Back_Mask = Back_Subt.apply(thresh)
Back_Mask_Two = Back_Subt.apply(thresh_Two)
Back_Mask_Three = Back_Subt.apply(thresh_Three)
Back_Mask_Four = Back_Subt.apply(thresh_Four)

fig, axes = plt.subplots(nrows=2,
                         ncols=2,
                         figsize=(20, 20),
                        subplot_kw={'xticks': [], 'yticks': []})

axes[0,0].imshow(Back_Mask)
axes[0,1].imshow(Back_Mask_Two)
axes[1,0].imshow(Back_Mask_Three)
axes[1,1].imshow(Back_Mask_Four)

# MODELING PROCESS

In [ ]:
Back_Subt = cv2.createBackgroundSubtractorMOG2()
New_PNG_List = []
img_size= (160, 160)

for im in Main_Train_Data["PNG"].values:
    Read_IMG_New = cv2.imread(im,0)
    Thresh_IMG = cv2.adaptiveThreshold(Read_IMG_New,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)
    Mask_IMG = Back_Subt.apply(Thresh_IMG)
    Mask_IMG = cv2.resize(Mask_IMG,img_size,interpolation=cv2.INTER_AREA)
    New_PNG_List.append(Mask_IMG/255.0)

In [ ]:
New_PNG_List = np.stack(New_PNG_List)

In [ ]:
print(type(New_PNG_List))

In [ ]:
print(New_PNG_List.shape)

In [ ]:
print(New_PNG_List[5].shape)

In [ ]:
encode = LabelEncoder()
Main_Train_Data["CATEGORY"] = encode.fit_transform(Main_Train_Data["CATEGORY"])

In [ ]:
New_Labels_List = []
for label in Main_Train_Data["CATEGORY"].values:
    New_Labels_List.append(label)

In [ ]:
New_Labels_List = np.asarray(New_Labels_List)

In [ ]:
print(type(New_Labels_List))

In [ ]:
print(New_Labels_List.shape)

In [ ]:
print(New_Labels_List[777])

In [ ]:
xTrain,xTest,yTrain,yTest = train_test_split(New_PNG_List,New_Labels_List,
                                             train_size=0.8,
                                             shuffle=True,
                                             random_state=42)

In [ ]:
print(xTrain.shape)

In [ ]:
print(xTrain.shape)

In [ ]:
sample_shape = xTrain[0].shape
img_width, img_height = sample_shape[0], sample_shape[1]
input_shape = (img_width, img_height, 1)

In [ ]:
xTrain = xTrain.reshape(len(xTrain),input_shape[0],input_shape[1],input_shape[2])

In [ ]:
print(xTrain.shape)

In [ ]:
print(yTrain.shape)

In [ ]:
yTrain = to_categorical(yTrain)

In [ ]:
xTest = xTest.reshape(len(xTest),input_shape[0],input_shape[1],input_shape[2])

In [ ]:
print(xTest.shape)

In [ ]:
print(yTest.shape)

In [ ]:
yTest = to_categorical(yTest)

In [ ]:
xTrain = xTrain / 255

In [ ]:
xTest = xTest / 255

#### MODEL I

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3,3), activation='relu',input_shape=(160,160,1)))
model.add(Conv2D(64, (3,3), activation='relu'))

model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(Conv2D(128, (3,3), activation='relu',padding="same"))
model.add(Conv2D(128, (3,3), activation='relu',padding="same"))

model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(Conv2D(256, (3,3), activation='relu',padding="same"))
model.add(Conv2D(256, (3,3), activation='relu',padding="same"))
model.add(Conv2D(256, (3,3), activation='relu',padding="same"))

model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Conv2D(512, (3,3), activation='relu',padding="same"))
model.add(Conv2D(512, (3,3), activation='relu',padding="same"))
model.add(Conv2D(512, (3,3), activation='relu',padding="same"))

model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Conv2D(512, (3,3), activation='relu',padding="same"))
model.add(Conv2D(512, (3,3), activation='relu',padding="same"))
model.add(Conv2D(512, (3,3), activation='relu',padding="same"))

model.add(MaxPooling2D((2,2), strides=(2,2)))


model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(21,activation="softmax"))

In [ ]:
print(model.summary())

In [ ]:
plot_model(model,to_file="Model_One.png",show_layer_names=True,show_dtype=True,show_shapes=True)

#### IMAGE GENERATOR FOR MODEL II

In [ ]:
Train_Generator = ImageDataGenerator(rescale=1./255,
                                    shear_range=0.3,
                                    zoom_range=0.2,
                                    brightness_range=[0.2,0.9],
                                    rotation_range=30,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    fill_mode="nearest",
                                    validation_split=0.1)

In [ ]:
Test_Generator = ImageDataGenerator(rescale=1./255,validation_split=0.1)

In [ ]:
Train_PNG_Path_Series = pd.Series(Train_PNG_Path,name="PNG").astype(str)
Test_PNG_Path_Series = pd.Series(Test_PNG_Path,name="PNG").astype(str)
Validation_PNG_Path_Series = pd.Series(Validation_PNG_Path,name="PNG").astype(str)
Train_PNG_Labels_Series = pd.Series(Train_PNG_Labels,name="CATEGORY")
Test_PNG_Labels_Series = pd.Series(Test_PNG_Labels,name="CATEGORY")
Validation_PNG_Labels_Series = pd.Series(Validation_PNG_Labels,name="CATEGORY")

In [ ]:
Main_Train_Data = pd.concat([Train_PNG_Path_Series,Train_PNG_Labels_Series],axis=1)
Main_Test_Data = pd.concat([Test_PNG_Path_Series,Test_PNG_Labels_Series],axis=1)
Main_Validation_Data = pd.concat([Validation_PNG_Path_Series,Validation_PNG_Labels_Series],axis=1)

In [ ]:
Main_Train_Data = Main_Train_Data.sample(frac=1).reset_index(drop=True)
Main_Test_Data = Main_Test_Data.sample(frac=1).reset_index(drop=True)
Main_Validation_Data = Main_Validation_Data.sample(frac=1).reset_index(drop=True)

In [ ]:
Train_IMG_Set = Train_Generator.flow_from_dataframe(dataframe=Main_Train_Data,
                                                   x_col="PNG",
                                                   y_col="CATEGORY",
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   batch_size=32,
                                                   subset="training")

In [ ]:
Test_IMG_Set = Test_Generator.flow_from_dataframe(dataframe=Main_Test_Data,
                                                   x_col="PNG",
                                                   y_col="CATEGORY",
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   batch_size=32)

In [ ]:
Validation_IMG_Set = Test_Generator.flow_from_dataframe(dataframe=Main_Validation_Data,
                                                   x_col="PNG",
                                                   y_col="CATEGORY",
                                                   color_mode="rgb",
                                                   class_mode="categorical",
                                                   batch_size=32,
                                                   subset="validation")

In [ ]:
for data_batch,label_batch in Train_IMG_Set:
    print("DATA SHAPE: ",data_batch.shape)
    print("LABEL SHAPE: ",label_batch.shape)
    break

In [ ]:
for data_batch,label_batch in Validation_IMG_Set:
    print("DATA SHAPE: ",data_batch.shape)
    print("LABEL SHAPE: ",label_batch.shape)
    break

In [ ]:
for data_batch,label_batch in Test_IMG_Set:
    print("DATA SHAPE: ",data_batch.shape)
    print("LABEL SHAPE: ",label_batch.shape)
    break

In [ ]:
print("TRAIN: ")
print(Train_IMG_Set.class_indices)
print(Train_IMG_Set.classes[0:5])
print(Train_IMG_Set.image_shape)
print("---"*20)
print("VALIDATION: ")
print(Validation_IMG_Set.class_indices)
print(Validation_IMG_Set.classes[0:5])
print(Validation_IMG_Set.image_shape)
print("---"*20)
print("TEST: ")
print(Test_IMG_Set.class_indices)
print(Test_IMG_Set.classes[0:5])
print(Test_IMG_Set.image_shape)

#### MODEL II

In [ ]:
Model = Sequential()

Model.add(SeparableConv2D(32,3,
                          activation="relu",
                 input_shape=(226,226,3)))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2)))

#
Model.add(SeparableConv2D(64,3,
                 activation="relu"))
Model.add(SeparableConv2D(128,(3,3),
                 activation="relu"))
Model.add(Dropout(0.5))
Model.add(MaxPooling2D((2)))

#
Model.add(SeparableConv2D(128,3,
                 activation="relu"))
Model.add(SeparableConv2D(128,3,
                 activation="relu"))
Model.add(Dropout(0.5))
Model.add(GlobalAveragePooling2D())


#
Model.add(Flatten())
Model.add(Dense(256,
                activation="relu"))
Model.add(Dropout(0.5))
Model.add(Dense(21,
                activation="softmax"))

In [ ]:
Call_Back = tf.keras.callbacks.EarlyStopping(monitor="loss",patience=5,mode="min")

In [ ]:
Model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

CNN_Model = Model.fit(Train_IMG_Set,
                      validation_data=Validation_IMG_Set,
                      callbacks=Call_Back,
                      epochs=100)

In [ ]:
print(Model.summary())

In [ ]:
plot_model(Model,to_file="Model_Two.png",show_layer_names=True,show_dtype=True,show_shapes=True)

In [ ]:
Model_Results = Model.evaluate(Test_IMG_Set)
print("LOSS:  " + "%.4f" % Model_Results[0])
print("ACCURACY:  " + "%.2f" % Model_Results[1])

In [ ]:
plt.plot(CNN_Model.history["accuracy"])
plt.plot(CNN_Model.history["val_accuracy"])
plt.ylabel("ACCURACY")
plt.legend()
plt.show()

In [ ]:
plt.plot(CNN_Model.history["loss"])
plt.plot(CNN_Model.history["val_loss"])
plt.ylabel("LOSS")
plt.legend()
plt.show()

In [ ]:
plt.plot(CNN_Model.history["val_accuracy"])
plt.plot(CNN_Model.history["val_loss"])
plt.ylabel("ACCURACY-LOSS")
plt.legend()
plt.show()

In [ ]:
Dict_Summary_One = pd.DataFrame(CNN_Model.history)
Dict_Summary_One.plot()